<a href="https://colab.research.google.com/github/arti1117/hg-mldl/blob/main/05_2_Cross_Validation_and_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")

data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

In [44]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, random_state=42, test_size=.2
)

In [45]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=.2, random_state=42
)

In [46]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [47]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [48]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.013381  , 0.01182556, 0.0110352 , 0.0107913 , 0.01047397]), 'score_time': array([0.00126934, 0.00119972, 0.00131392, 0.0013299 , 0.00120187]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [49]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [50]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [51]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [52]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease': [.0001, .0002, .0003, .0004, .0005]
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_

print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [53]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [54]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [55]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [56]:
params = {
    'min_impurity_decrease': np.arange(.0001, .001, .0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10)
}

In [57]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731
0.892053107562055
0.8615384615384616


In [58]:
from scipy.stats import uniform, randint

In [59]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 0, 4, 3, 0, 0, 5, 3, 0, 9])

In [60]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([121,  98,  91, 101,  97, 102, 110,  95, 103,  82]))

In [61]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.06334511, 0.6437912 , 0.72076338, 0.49833583, 0.81144368,
       0.2342943 , 0.6588656 , 0.9055375 , 0.91835021, 0.04223636])

In [62]:
np.unique(ugen.rvs(1000), return_counts=True)

(array([8.72630235e-04, 1.19908018e-03, 2.07919943e-03, 4.81442192e-03,
        5.25299849e-03, 6.42251233e-03, 8.54515034e-03, 9.49653680e-03,
        1.39006433e-02, 1.41728935e-02, 1.42039319e-02, 1.51088517e-02,
        1.52056784e-02, 1.67239184e-02, 1.71460418e-02, 1.85008303e-02,
        1.90516059e-02, 1.99809333e-02, 2.04061659e-02, 2.23767438e-02,
        2.32839356e-02, 2.43420155e-02, 2.74394060e-02, 2.80684587e-02,
        2.86959177e-02, 2.91323619e-02, 3.01391745e-02, 3.07932008e-02,
        3.17416953e-02, 3.20207008e-02, 3.35268500e-02, 3.42649417e-02,
        3.62051058e-02, 3.73772033e-02, 3.75786915e-02, 3.85888211e-02,
        4.06343313e-02, 4.09349606e-02, 4.09655237e-02, 4.19036593e-02,
        4.24771535e-02, 4.26191553e-02, 4.33102661e-02, 4.34457308e-02,
        4.52655160e-02, 4.53853958e-02, 4.53888824e-02, 4.57257188e-02,
        4.77626508e-02, 4.82545065e-02, 4.83434440e-02, 5.16229042e-02,
        5.44225512e-02, 5.46280381e-02, 5.65305348e-02, 5.700338

In [63]:
params = {
    'min_impurity_decrease': uniform(.0001, .001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25),
}

In [64]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb52963790>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb529639d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb5277b6d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb5277ba90>},
                   random_state=42)

In [65]:
print(rs.best_params_)
print(np.max(rs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [66]:
dt = rs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.8928227823744468
0.86


In [67]:
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'),
                        params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

print(rs.best_params_)
print(np.max(rs.cv_results_['mean_test_score']))

dt = rs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.8043101789493938
0.786923076923077
